# SFM PIPELINE

In [ ]:
from sfm import SFM

# CHANGE TO IMAGE SOURCE FILE (and db path for pycolmap)
img_src = ""
db_path = "database.db"

## 1. FEATURE EXTRACTION AND MATCHING:
Flag options for BRISK-SIFT/SIFT-SIFT and image preprocessing (for obj. isolation)\

BRISK-SIFT (`brisk=True`)
- recommended when object has lots of texture

SIFT-SIFT (`brisk=False`)
- recommended when object has low texture; able to detect more 'subtle' features at the expense of computation

In [ ]:
sfm = SFM(img_src, db_path=db_path, preprocess=False, brisk=True)

sfm.ft_extract()
sfm.ft_match()

## 2. SFM cont'd (3 options)
(Triangulation, Bundle Adjustment, Sparse/Dense Reconstruction)
1. COLMAP GUI
2. PyCOLMAP
3. PyCOLMAP and COLMAP GUI 

**note:** PyCOLMAP (on ft matches) is very inconsistent

### 2.1 via COLMAP GUI **recommended**
Create a colmap project in `img_src` and import features + feature matches. Continue the reconstruction pipeline from there\

ie. import features (via img path) &rarr; import matches (via match.txt for inlier feature matches) &rarr; start reconstruction &rarr; \
dense: undistort &rarr; stereo &rarr; fusion

In [ ]:
# create .txt files for image features + matches
sfm.output()
# import these into COLMAP

### 2.2 via PyCOLMAP (**PYCOLMAP MUST BE COMPILED WITH CUDA**, ie. built from source)

In [ ]:
# SET DESTINATION FOLDER AND OBJECT NAME
dest = "output" 
obj_name = "object"
sfm.reconstruction_sparse(dest=dest, obj_name=obj_name)
sfm.reconstruction_dense(dest=dest, model_folder='/0') # can also change model_folder to '\1'


### 2.3a via PyCOLMAP database (contains features + cams only) and COLMAP
**recommended**

In [ ]:
dest = "output" 
obj_name = "object"
sfm.add_cams()
sfm.add_ft()
sfm.output()
# Use .db file + match.txt (inlier matches) in COLMAP GUI

### 2.3b via PyCOLMAP (sparse reconstruction) and COLMAP (dense reconstruction)
Inconsistent for some datasets

In [ ]:
dest = "output" 
obj_name = "object"
sfm.reconstruction_sparse(dest=dest, obj_name=obj_name)
# Use .db file in COLMAP GUI for dense reconstruction

# Surface/Mesh Reconstruction
Use resulting dense point cloud .ply files in the [surface reconstruction pipeline](reconstruction_pipeline.ipynb)